In [ ]:
import argparse
import os
import glob
from random import shuffle
import numpy as np
from matplotlib import cm
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

from lib.NmnistClass import NmnistDataset
from lib.feature import remove_isolated_pixels
from lib.layer import initialise_time_surface_prototypes


In [ ]:
N_1 = 4
tau_1 = 20000.
r_1 = 2

K_N = 2
K_tau = 2
K_r = 2

N_2 = N_1 * K_N
tau_2 = tau_1 * K_tau
r_2 = r_1 * K_r

N_3 = N_2 * K_N
tau_3 = tau_2 * K_tau
r_3 = r_2 * K_r


In [ ]:
input_folders_training = [
    'datasets/mnist/Train/0',
    'datasets/mnist/Train/1',
    'datasets/mnist/Train/2',
    'datasets/mnist/Train/3',
    'datasets/mnist/Train/4',
    'datasets/mnist/Train/5',
    'datasets/mnist/Train/6',
    'datasets/mnist/Train/7',
    'datasets/mnist/Train/8',
    'datasets/mnist/Train/9']

num_files_per_folder = 4

input_files_all = []

for folder in input_folders_training:
    input_files = glob.glob(os.path.join(folder, '*.bin'))[:num_files_per_folder]
    input_files_all.extend(input_files)
    print('Num files from {}: {}'.format(folder, len(input_files)))


In [ ]:
shuffle(input_files_all)

ev = NmnistDataset.read_dataset(input_files_all[0])

# filter out outliers

event_data = []
event_data_filt = []

for f in input_files_all:
    ev_data = NmnistDataset.read_dataset(f).data
    ev_data_filt = remove_isolated_pixels(ev_data, eps=3, min_samples=20)[0]

    if len(event_data) > 0:
        ts_start_0 = event_data[-1].ts

        for i in range(len(ev_data)):
            ev_data[i].ts += ts_start_0

    if len(event_data_filt) > 0:
        ts_start_1 = event_data_filt[-1].ts

        for i in range(len(ev_data_filt)):
            ev_data_filt[i].ts += ts_start_1

    event_data.extend(ev_data)
    event_data_filt.extend(ev_data_filt)

    print('length event stream:', len(ev_data), len(ev_data_filt))


In [ ]:
C_1 = initialise_time_surface_prototypes(N_1, tau_1, r_1, ev.width, ev.height, event_data_filt, plot=True)

In [ ]:
train_layer(C_1, N_1, tau_1, r_1, ev.width, ev.height, event_data_filt, num_polarities=2, layer_number=1, plot=True)